# HadGEM3-GC31_25km Seasonal SST GeoTIFF

In [2]:
import xarray as xr
import rioxarray
import numpy as np
import netCDF4
import os
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as mpl_colors
import math
import geopandas as gpd
import calendar

ds = xr.open_dataset(r"E:\Script\MSP_Data_office_PC\CMIP6\Data\HadGEM3-GC31-MM_ssp585_r1i1p1f3_gn_25km.nc")
ouput_folder_path = r"E:\Script\MSP_Data_office_PC\CMIP6\Output"

latitude_boundary = [16, 24]
longitude_boundary = [81, 95]

llc_lat_idx = np.argmin(np.abs(ds['latitude'][:,1].data - latitude_boundary[0]))
lrc_lat_idx = np.argmin(np.abs(ds['latitude'][:,1].data - latitude_boundary[1]))

llc_lon_idx = np.argmin(np.abs(ds['longitude'][1,:].data - longitude_boundary[0]))
lrc_lon_idx = np.argmin(np.abs(ds['longitude'][1,:].data - longitude_boundary[1]))


winter_months = [time.data.tolist() for time in ds.time if time.dt.month.data in [1, 2, 12] and time.dt.year.data == 2070]
summer_months = [time.data.tolist() for time in ds.time if time.dt.month.data in [6, 7, 8] and time.dt.year.data == 2070]

season_dict = {'Winter': winter_months, 'Summer': summer_months}

season_ds_list = []

for season in season_dict.keys():
    
    season_ds = ds.sel(time = season_dict[season],
                       i = slice(ds['i'][llc_lon_idx].data, ds['i'][lrc_lon_idx].data),
                       j = slice(ds['j'][llc_lat_idx].data, ds['j'][lrc_lat_idx].data))
    
    season_mean_ds = season_ds.mean(dim= ['time'], skipna=True, keep_attrs=True)

    season_array = np.array([season], dtype = str)
    
    for var in season_mean_ds.data_vars:
        season_mean_ds[var] = season_mean_ds[var].expand_dims(season=season_array, axis=0)
        
    ds_name = '{}_ds'.format(season)
    ds_name = season_mean_ds
    season_ds_list.append(ds_name)

combined_sesason_ds = xr.concat(season_ds_list , dim = 'season')
season_name_array = combined_sesason_ds['season'].to_numpy()

combined_sesason_ds.coords['j'] = combined_sesason_ds['latitude'][:,0].data
combined_sesason_ds.coords['i'] = combined_sesason_ds['longitude'][0,:].data

for i, season_name in zip(range(0, len(combined_sesason_ds['season'])), season_name_array):
    
    season_temp_ds = combined_sesason_ds['tos'][i,:,:]
    
    season_temp_ds.rio.write_crs("epsg:4326", inplace=True)
    season_temp_ds.rio.write_nodata(-9999, encoded=True, inplace=True)

    season_temp_ds.rio.set_spatial_dims("i", "j", inplace = True)

    season_temp_ds.rio.to_raster(os.path.join(ouput_folder_path, 'Sea_Surface_Temp_{}_2070.tif'.format(season_name)))